<a href="https://colab.research.google.com/github/ArtyomIT/Interface-for-CatORdoG-model/blob/main/CatORdoG_with_Interface_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

In [1]:
!pip install openai gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 332.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.7/336.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.6/318.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/13

In [5]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from PIL import Image, ExifTags
import gradio as gr
import os

In [6]:
print(keras.__version__)

3.4.1


In [7]:
print(tf.__version__)

2.17.0


## Loading Model

In [9]:
!wget 'https://storage.yandexcloud.net/bucket-picture/model_catVSdog.keras'

--2024-08-06 14:49:39--  https://storage.yandexcloud.net/bucket-picture/model_catVSdog.keras
Resolving storage.yandexcloud.net (storage.yandexcloud.net)... 213.180.193.243, 2a02:6b8::1d9
Connecting to storage.yandexcloud.net (storage.yandexcloud.net)|213.180.193.243|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 31048634 (30M) [application/x-www-form-urlencoded]
Saving to: ‘model_catVSdog.keras’

model_catVSdog.kera 100%[===================>]  29.61M  11.9MB/s    in 2.5s    

2024-08-06 14:49:43 (11.9 MB/s) - ‘model_catVSdog.keras’ saved [31048634/31048634]



In [11]:
model = keras.models.load_model('model_catVSdog.keras')

## Base Version

In [12]:
def run_inference(image):
    if model is None:
        return "Model is not trained yet!"

    img = load_img(image, target_size=(150, 150))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    dog_probability = predictions[0][1]
    cat_probability = predictions[0][0]

    if dog_probability > cat_probability:
        result = f"The picture shows a DOG with a {dog_probability:.0%} chance"
    else:
        result = f"The picture shows a CAT with a {cat_probability:.0%} chance"

    return f"Predictions: {result}"

In [14]:
inference_interface = gr.Interface(
    fn=run_inference,
    inputs=gr.Image(type="filepath"),
    outputs="text",
    title='Please upload the image and then click the "Submit" button to check the model'
)

In [15]:
inference_interface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2503b5bbee1e01cb5c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


## Base version + info about image

In [16]:
global_image_info = {'path': None, 'info': None, 'metadata': None}

In [30]:
def run_inference(image):
    if model is None:
        return "Model is not trained yet!"

    img = load_img(image, target_size=(150, 150))
    plt.imshow(img)
    plt.axis('off')
    plt.show()

    img_array = img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0) / 255.0

    predictions = model.predict(img_array)
    dog_probability = predictions[0][1]
    cat_probability = predictions[0][0]

    if dog_probability > cat_probability:
        result = f"DOG with a {dog_probability:.0%} chance"
    else:
        result = f"CAT with a {cat_probability:.0%} chance"

    original_img = Image.open(image)
    info = {
        'width': original_img.width,
        'height': original_img.height,
        'format': original_img.format,
        'size': os.path.getsize(image)
    }

    metadata = {}
    if hasattr(original_img, '_getexif'):
        exifdata = original_img._getexif()
        if exifdata is not None:
            for tag, value in exifdata.items():
                decoded_tag = ExifTags.TAGS.get(tag, tag)
                if decoded_tag == 'GPSInfo' and isinstance(value, dict):
                    gps_info = {}
                    for gps_tag, gps_value in value.items():
                        decoded_gps_tag = ExifTags.GPSTAGS.get(gps_tag, gps_tag)
                        gps_info[decoded_gps_tag] = gps_value
                    metadata['GPSInfo'] = gps_info
                else:
                    metadata[decoded_tag] = value

    global_image_info['path'] = image
    global_image_info['info'] = info
    global_image_info['metadata'] = metadata

    return f"Predictions: {result}"

In [25]:
def img_info():
    global global_image_info
    if global_image_info['path'] is None:
        return "No image loaded."
    else:
        info = global_image_info['info']
        metadata = global_image_info['metadata']
        metadata_str = "\n".join([f"{key}: {value}" for key, value in metadata.items()])

        return f"Path: {global_image_info['path']}\nWidth: {info['width']}\nHeight: {info['height']}\nFormat: {info['format']}\nSize: {info['size']} байт\n\nMetadata:\n{metadata_str}"

In [26]:
info_interface = gr.Interface(
    fn=img_info,
    inputs=[],
    outputs="text",
    title='To view information about the image, please click the "Generate" button'
)

inference_interface = gr.Interface(
    fn=run_inference,
    inputs=gr.Image(type="filepath"),
    outputs="text",
    title='Please upload the image and then click the "Submit" button to check the model'
)

In [31]:
demo = gr.TabbedInterface([inference_interface, info_interface], ["Inference", "Info"])
demo.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://d35d870bf0b1630cb1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
